In [4]:
# Set seed for reproducibility
SEED = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
#import torch
#torch.manual_seed(SEED)
#from torch import nn
#from torchsummary import summary
#from torch.utils.tensorboard import SummaryWriter
#import torchvision
#from torchvision.transforms import v2 as transforms
#from torch.utils.data import TensorDataset, DataLoader
#!pip install torchview
#from torchview import draw_graph

# Configurazione di TensorBoard e directory
#logs_dir = "tensorboard"
#!pkill -f tensorboard
#%load_ext tensorboard
#!mkdir -p models

#if torch.cuda.is_available():
 #   device = torch.device("cuda")
  #  torch.cuda.manual_seed_all(SEED)
   # torch.backends.cudnn.benchmark = True
#else:
#    device = torch.device("cpu")

#print(f"PyTorch version: {torch.__version__}")
#print(f"Device: {device}")

# Import other libraries
import cv2
import copy
import shutil
from itertools import product
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.gridspec as gridspec
#import requests
from io import BytesIO

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

In [7]:

import cv2
import numpy as np

def analyze_image_sizes(folder):
    heights = []
    widths = []
    
    print(f"Analisi dimensioni in: {folder} ...")
    for filename in os.listdir(folder):
        # Leggi solo l'header dell'immagine per velocità (se possibile) o carica intera
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            h, w, _ = img.shape
            heights.append(h)
            widths.append(w)
            
    if not heights:
        print("Nessuna immagine trovata.")
        return

    avg_h = np.mean(heights)
    avg_w = np.mean(widths)
    median_h = np.median(heights)
    median_w = np.median(widths)
    
    print(f"--- Statistiche Immagini ---")
    print(f"Totale immagini: {len(heights)}")
    print(f"Media (H x W):   {avg_h:.1f} x {avg_w:.1f}")
    print(f"Mediana (H x W): {median_h:.1f} x {median_w:.1f}")
    print(f"Minima: {np.min(heights)} x {np.min(widths)}")
    print(f"Massima: {np.max(heights)} x {np.max(widths)}")

# Esegui l'analisi
analyze_image_sizes('../data/processed/images/')

Analisi dimensioni in: ../data/processed/images/ ...
--- Statistiche Immagini ---
Totale immagini: 1261
Media (H x W):   1331.6 x 1250.6
Mediana (H x W): 1082.0 x 1024.0
Minima: 1024 x 1024
Massima: 4233 x 3970


In [9]:
def load_images_from_folder(folder, labels_path=None):
    """
    Load and preprocess images from a specified folder.

    Args:
        folder (str): Path to the folder containing images
        labels_path (str, optional): Path to the CSV file containing labels. Defaults to None.

    Returns:
        np.ndarray: Array of preprocessed images with shape (N, H, W, C)
    """
    images = []
    labels = []
    if labels_path is not None:
            df_labels = pd.read_csv(labels_path)
    # Iterate through files in the specified folder
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))

        # Normalize image pixel values to a float range [0, 1]
        img = (img / 255).astype(np.float32)

        # Convert image from BGR to RGB
        img = img[...,::-1]
        
        # Resize the image to 1024x1024 pixels
        img = cv2.resize(img, (1024, 1024))
        
        label = df_labels[df_labels['sample_index'] == filename]['label'].values[0] if labels_path is not None else None
        print(f"Filename: {filename}, Label: {label}")
        label_mapping = {
            'Luminal A': 0,
            'Luminal B': 1,
            'HER2(+)': 2,
            'Triple negative': 3
        }
        labels.append(label_mapping[label]) if label is not None else None
        if img is not None:
            images.append(img)

    return np.array(images), np.array(labels)

# Load images from the 'animals/' folder
images_path = '../data/processed/images/'
mask_path = '../data/processed/masks/'
labels_path = '../data/processed/train_labels_processed.csv'
images, labels = load_images_from_folder(images_path, labels_path=labels_path)
mask, _ = load_images_from_folder(mask_path)

#print the shape of images and labels
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Masks shape: {mask.shape}")

Filename: img_0564.png, Label: Luminal B
Filename: img_0202.png, Label: Luminal B
Filename: img_0216.png, Label: Luminal B
Filename: img_1108.png, Label: Triple negative
Filename: img_0570.png, Label: Luminal B
Filename: img_0558.png, Label: HER2(+)
Filename: img_1120.png, Label: HER2(+)
Filename: img_1134.png, Label: Luminal A
Filename: img_0000.png, Label: Triple negative
Filename: img_0766.png, Label: HER2(+)
Filename: img_0772.png, Label: Luminal B
Filename: img_0014.png, Label: Luminal A
Filename: img_1322.png, Label: Luminal A
Filename: img_0982.png, Label: HER2(+)
Filename: img_0996.png, Label: Luminal B
Filename: img_1336.png, Label: HER2(+)
Filename: img_0028.png, Label: HER2(+)
Filename: img_0969.png, Label: Luminal A
Filename: img_0941.png, Label: Luminal A
Filename: img_0799.png, Label: Luminal B
Filename: img_0955.png, Label: Triple negative
Filename: img_0174.png, Label: Luminal A
Filename: img_0612.png, Label: Luminal A
Filename: img_0160.png, Label: Triple negative
File

Labels: [1 1 1 ... 1 2 0]
